# Music-Recommender System
##**General outline:**

###Hack To Get Data
*   Before using, adjust this file to your file hierachy under **Hack to get Data/Local storage**
*   The msd subset is stored as hfd5 files, **HDF5 Getters (imported) and necessary function** contains all functions to process those
*   Two individual datasets are created

> * one non-audio set (containing features like artist etc.)
>> * **Extract non audio**
> * one audio set (containing musical features)
>> * **Extract audio**
* Since already the subset takes some time to be processed. 'smallset' is a subset of the subset containing 53 files. Whenever the hfd5 files are extracted, use *smallset* instead of *msd_subset_data_path* to extract if you want quick results
* Pandas Dataframe for non Audio: *nonAudioData*
* Pandas Dataframe for  Audio: *audioData*






In [140]:
!pip install tables

##Hack to get the data 

In [0]:
import os
import sys
import time
import glob
import datetime
import sqlite3
import numpy as np
import tarfile
import tables
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from google.colab import drive

**Local Storage**

In [142]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [143]:
msd_subset_path=r'/content/drive/My Drive/IR/Dataprocessing/MillionSongSubset'
msd_subset_data_path=os.path.join(msd_subset_path,'data')
msd_subset_addf_path=os.path.join(msd_subset_path,'AdditionalFiles')
smallset = r'/content/drive/My Drive/IR/Dataprocessing/A'
msd_code_path=r'/content/drive/My Drive/IR/Dataprocessing'
sys.path.append( os.path.join(msd_code_path,'PythonSrc') )
sys.path.append( os.path.join(msd_code_path,'PythonSrc') )

#Is the given path really a path?
print(os.path.isdir(msd_subset_path))
print(os.path.exists(msd_subset_path))
print(os.path.isdir(msd_code_path))
print(os.path.exists(msd_code_path))
print(os.path.isdir(smallset))

True
True
True
True
True


###HDF5 Getters (imported) and necessary function

In [0]:
"""
Thierry Bertin-Mahieux (2010) Columbia University
tb2332@columbia.edu
This code contains a set of getters functions to access the fields
from an HDF5 song file (regular file with one song or
aggregate / summary file with many songs)
This is part of the Million Song Dataset project from
LabROSA (Columbia University) and The Echo Nest.
Copyright 2010, Thierry Bertin-Mahieux
This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.
You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
"""


def open_h5_file_read(h5filename):
    """
    Open an existing H5 in read mode.
    Same function as in hdf5_utils, here so we avoid one import
    """
    return tables.open_file(h5filename, mode='r')


def get_num_songs(h5):
    """
    Return the number of songs contained in this h5 file, i.e. the number of rows
    for all basic informations like name, artist, ...
    """
    return h5.root.metadata.songs.nrows

def get_artist_familiarity(h5,songidx=0):
    """
    Get artist familiarity from a HDF5 song file, by default the first song in it
    """
    return h5.root.metadata.songs.cols.artist_familiarity[songidx]

def get_artist_hotttnesss(h5,songidx=0):
    """
    Get artist hotttnesss from a HDF5 song file, by default the first song in it
    """
    return h5.root.metadata.songs.cols.artist_hotttnesss[songidx]

def get_artist_id(h5,songidx=0):
    """
    Get artist id from a HDF5 song file, by default the first song in it
    """
    return h5.root.metadata.songs.cols.artist_id[songidx]

def get_artist_mbid(h5,songidx=0):
    """
    Get artist musibrainz id from a HDF5 song file, by default the first song in it
    """
    return h5.root.metadata.songs.cols.artist_mbid[songidx]

def get_artist_playmeid(h5,songidx=0):
    """
    Get artist playme id from a HDF5 song file, by default the first song in it
    """
    return h5.root.metadata.songs.cols.artist_playmeid[songidx]

def get_artist_7digitalid(h5,songidx=0):
    """
    Get artist 7digital id from a HDF5 song file, by default the first song in it
    """
    return h5.root.metadata.songs.cols.artist_7digitalid[songidx]

def get_artist_latitude(h5,songidx=0):
    """
    Get artist latitude from a HDF5 song file, by default the first song in it
    """
    return h5.root.metadata.songs.cols.artist_latitude[songidx]

def get_artist_longitude(h5,songidx=0):
    """
    Get artist longitude from a HDF5 song file, by default the first song in it
    """
    return h5.root.metadata.songs.cols.artist_longitude[songidx]

def get_artist_location(h5,songidx=0):
    """
    Get artist location from a HDF5 song file, by default the first song in it
    """
    return h5.root.metadata.songs.cols.artist_location[songidx]

def get_artist_name(h5,songidx=0):
    """
    Get artist name from a HDF5 song file, by default the first song in it
    """
    return h5.root.metadata.songs.cols.artist_name[songidx]

def get_release(h5,songidx=0):
    """
    Get release from a HDF5 song file, by default the first song in it
    """
    return h5.root.metadata.songs.cols.release[songidx]

def get_release_7digitalid(h5,songidx=0):
    """
    Get release 7digital id from a HDF5 song file, by default the first song in it
    """
    return h5.root.metadata.songs.cols.release_7digitalid[songidx]

def get_song_id(h5,songidx=0):
    """
    Get song id from a HDF5 song file, by default the first song in it
    """
    return h5.root.metadata.songs.cols.song_id[songidx]

def get_song_hotttnesss(h5,songidx=0):
    """
    Get song hotttnesss from a HDF5 song file, by default the first song in it
    """
    return h5.root.metadata.songs.cols.song_hotttnesss[songidx]

def get_title(h5,songidx=0):
    """
    Get title from a HDF5 song file, by default the first song in it
    """
    return h5.root.metadata.songs.cols.title[songidx]

def get_track_7digitalid(h5,songidx=0):
    """
    Get track 7digital id from a HDF5 song file, by default the first song in it
    """
    return h5.root.metadata.songs.cols.track_7digitalid[songidx]

def get_similar_artists(h5,songidx=0):
    """
    Get similar artists array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.metadata.songs.nrows == songidx + 1:
        return h5.root.metadata.similar_artists[h5.root.metadata.songs.cols.idx_similar_artists[songidx]:]
    return h5.root.metadata.similar_artists[h5.root.metadata.songs.cols.idx_similar_artists[songidx]:
                                            h5.root.metadata.songs.cols.idx_similar_artists[songidx+1]]

def get_artist_terms(h5,songidx=0):
    """
    Get artist terms array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.metadata.songs.nrows == songidx + 1:
        return h5.root.metadata.artist_terms[h5.root.metadata.songs.cols.idx_artist_terms[songidx]:]
    return h5.root.metadata.artist_terms[h5.root.metadata.songs.cols.idx_artist_terms[songidx]:
                                            h5.root.metadata.songs.cols.idx_artist_terms[songidx+1]]

def get_artist_terms_freq(h5,songidx=0):
    """
    Get artist terms array frequencies. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.metadata.songs.nrows == songidx + 1:
        return h5.root.metadata.artist_terms_freq[h5.root.metadata.songs.cols.idx_artist_terms[songidx]:]
    return h5.root.metadata.artist_terms_freq[h5.root.metadata.songs.cols.idx_artist_terms[songidx]:
                                              h5.root.metadata.songs.cols.idx_artist_terms[songidx+1]]

def get_artist_terms_weight(h5,songidx=0):
    """
    Get artist terms array frequencies. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.metadata.songs.nrows == songidx + 1:
        return h5.root.metadata.artist_terms_weight[h5.root.metadata.songs.cols.idx_artist_terms[songidx]:]
    return h5.root.metadata.artist_terms_weight[h5.root.metadata.songs.cols.idx_artist_terms[songidx]:
                                                h5.root.metadata.songs.cols.idx_artist_terms[songidx+1]]

def get_analysis_sample_rate(h5,songidx=0):
    """
    Get analysis sample rate from a HDF5 song file, by default the first song in it
    """
    return h5.root.analysis.songs.cols.analysis_sample_rate[songidx]

def get_audio_md5(h5,songidx=0):
    """
    Get audio MD5 from a HDF5 song file, by default the first song in it
    """
    return h5.root.analysis.songs.cols.audio_md5[songidx]

def get_danceability(h5,songidx=0):
    """
    Get danceability from a HDF5 song file, by default the first song in it
    """
    return h5.root.analysis.songs.cols.danceability[songidx]

def get_duration(h5,songidx=0):
    """
    Get duration from a HDF5 song file, by default the first song in it
    """
    return h5.root.analysis.songs.cols.duration[songidx]

def get_end_of_fade_in(h5,songidx=0):
    """
    Get end of fade in from a HDF5 song file, by default the first song in it
    """
    return h5.root.analysis.songs.cols.end_of_fade_in[songidx]

def get_energy(h5,songidx=0):
    """
    Get energy from a HDF5 song file, by default the first song in it
    """
    return h5.root.analysis.songs.cols.energy[songidx]

def get_key(h5,songidx=0):
    """
    Get key from a HDF5 song file, by default the first song in it
    """
    return h5.root.analysis.songs.cols.key[songidx]

def get_key_confidence(h5,songidx=0):
    """
    Get key confidence from a HDF5 song file, by default the first song in it
    """
    return h5.root.analysis.songs.cols.key_confidence[songidx]

def get_loudness(h5,songidx=0):
    """
    Get loudness from a HDF5 song file, by default the first song in it
    """
    return h5.root.analysis.songs.cols.loudness[songidx]

def get_mode(h5,songidx=0):
    """
    Get mode from a HDF5 song file, by default the first song in it
    """
    return h5.root.analysis.songs.cols.mode[songidx]

def get_mode_confidence(h5,songidx=0):
    """
    Get mode confidence from a HDF5 song file, by default the first song in it
    """
    return h5.root.analysis.songs.cols.mode_confidence[songidx]

def get_start_of_fade_out(h5,songidx=0):
    """
    Get start of fade out from a HDF5 song file, by default the first song in it
    """
    return h5.root.analysis.songs.cols.start_of_fade_out[songidx]

def get_tempo(h5,songidx=0):
    """
    Get tempo from a HDF5 song file, by default the first song in it
    """
    return h5.root.analysis.songs.cols.tempo[songidx]

def get_time_signature(h5,songidx=0):
    """
    Get signature from a HDF5 song file, by default the first song in it
    """
    return h5.root.analysis.songs.cols.time_signature[songidx]

def get_time_signature_confidence(h5,songidx=0):
    """
    Get signature confidence from a HDF5 song file, by default the first song in it
    """
    return h5.root.analysis.songs.cols.time_signature_confidence[songidx]

def get_track_id(h5,songidx=0):
    """
    Get track id from a HDF5 song file, by default the first song in it
    """
    return h5.root.analysis.songs.cols.track_id[songidx]

def get_segments_start(h5,songidx=0):
    """
    Get segments start array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    h5 = os.path.normpath(h5)
    if h5.root.analysis.songs.nrows == songidx + 1:
        return h5.root.analysis.segments_start[h5.root.analysis.songs.cols.idx_segments_start[songidx]:]
    return h5.root.analysis.segments_start[h5.root.analysis.songs.cols.idx_segments_start[songidx]:
                                           h5.root.analysis.songs.cols.idx_segments_start[songidx+1]]
    
def get_segments_confidence(h5,songidx=0):
    """
    Get segments confidence array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.analysis.songs.nrows == songidx + 1:
        return h5.root.analysis.segments_confidence[h5.root.analysis.songs.cols.idx_segments_confidence[songidx]:]
    return h5.root.analysis.segments_confidence[h5.root.analysis.songs.cols.idx_segments_confidence[songidx]:
                                                h5.root.analysis.songs.cols.idx_segments_confidence[songidx+1]]

def get_segments_pitches(h5,songidx=0):
    """
    Get segments pitches array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.analysis.songs.nrows == songidx + 1:
        return h5.root.analysis.segments_pitches[h5.root.analysis.songs.cols.idx_segments_pitches[songidx]:,:]
    return h5.root.analysis.segments_pitches[h5.root.analysis.songs.cols.idx_segments_pitches[songidx]:
                                             h5.root.analysis.songs.cols.idx_segments_pitches[songidx+1],:]

def get_segments_timbre(h5,songidx=0):
    """
    Get segments timbre array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.analysis.songs.nrows == songidx + 1:
        return h5.root.analysis.segments_timbre[h5.root.analysis.songs.cols.idx_segments_timbre[songidx]:,:]
    return h5.root.analysis.segments_timbre[h5.root.analysis.songs.cols.idx_segments_timbre[songidx]:
                                            h5.root.analysis.songs.cols.idx_segments_timbre[songidx+1],:]

def get_segments_loudness_max(h5,songidx=0):
    """
    Get segments loudness max array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.analysis.songs.nrows == songidx + 1:
        return h5.root.analysis.segments_loudness_max[h5.root.analysis.songs.cols.idx_segments_loudness_max[songidx]:]
    return h5.root.analysis.segments_loudness_max[h5.root.analysis.songs.cols.idx_segments_loudness_max[songidx]:
                                                  h5.root.analysis.songs.cols.idx_segments_loudness_max[songidx+1]]

def get_segments_loudness_max_time(h5,songidx=0):
    """
    Get segments loudness max time array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.analysis.songs.nrows == songidx + 1:
        return h5.root.analysis.segments_loudness_max_time[h5.root.analysis.songs.cols.idx_segments_loudness_max_time[songidx]:]
    return h5.root.analysis.segments_loudness_max_time[h5.root.analysis.songs.cols.idx_segments_loudness_max_time[songidx]:
                                                       h5.root.analysis.songs.cols.idx_segments_loudness_max_time[songidx+1]]

def get_segments_loudness_start(h5,songidx=0):
    """
    Get segments loudness start array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.analysis.songs.nrows == songidx + 1:
        return h5.root.analysis.segments_loudness_start[h5.root.analysis.songs.cols.idx_segments_loudness_start[songidx]:]
    return h5.root.analysis.segments_loudness_start[h5.root.analysis.songs.cols.idx_segments_loudness_start[songidx]:
                                                    h5.root.analysis.songs.cols.idx_segments_loudness_start[songidx+1]]

def get_sections_start(h5,songidx=0):
    """
    Get sections start array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.analysis.songs.nrows == songidx + 1:
        return h5.root.analysis.sections_start[h5.root.analysis.songs.cols.idx_sections_start[songidx]:]
    return h5.root.analysis.sections_start[h5.root.analysis.songs.cols.idx_sections_start[songidx]:
                                           h5.root.analysis.songs.cols.idx_sections_start[songidx+1]]

def get_sections_confidence(h5,songidx=0):
    """
    Get sections confidence array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.analysis.songs.nrows == songidx + 1:
        return h5.root.analysis.sections_confidence[h5.root.analysis.songs.cols.idx_sections_confidence[songidx]:]
    return h5.root.analysis.sections_confidence[h5.root.analysis.songs.cols.idx_sections_confidence[songidx]:
                                                h5.root.analysis.songs.cols.idx_sections_confidence[songidx+1]]

def get_beats_start(h5,songidx=0):
    """
    Get beats start array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.analysis.songs.nrows == songidx + 1:
        return h5.root.analysis.beats_start[h5.root.analysis.songs.cols.idx_beats_start[songidx]:]
    return h5.root.analysis.beats_start[h5.root.analysis.songs.cols.idx_beats_start[songidx]:
                                        h5.root.analysis.songs.cols.idx_beats_start[songidx+1]]

def get_beats_confidence(h5,songidx=0):
    """
    Get beats confidence array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.analysis.songs.nrows == songidx + 1:
        return h5.root.analysis.beats_confidence[h5.root.analysis.songs.cols.idx_beats_confidence[songidx]:]
    return h5.root.analysis.beats_confidence[h5.root.analysis.songs.cols.idx_beats_confidence[songidx]:
                                             h5.root.analysis.songs.cols.idx_beats_confidence[songidx+1]]

def get_bars_start(h5,songidx=0):
    """
    Get bars start array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.analysis.songs.nrows == songidx + 1:
        return h5.root.analysis.bars_start[h5.root.analysis.songs.cols.idx_bars_start[songidx]:]
    return h5.root.analysis.bars_start[h5.root.analysis.songs.cols.idx_bars_start[songidx]:
                                       h5.root.analysis.songs.cols.idx_bars_start[songidx+1]]

def get_bars_confidence(h5,songidx=0):
    """
    Get bars start array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.analysis.songs.nrows == songidx + 1:
        return h5.root.analysis.bars_confidence[h5.root.analysis.songs.cols.idx_bars_confidence[songidx]:]
    return h5.root.analysis.bars_confidence[h5.root.analysis.songs.cols.idx_bars_confidence[songidx]:
                                            h5.root.analysis.songs.cols.idx_bars_confidence[songidx+1]]

def get_tatums_start(h5,songidx=0):
    """
    Get tatums start array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.analysis.songs.nrows == songidx + 1:
        return h5.root.analysis.tatums_start[h5.root.analysis.songs.cols.idx_tatums_start[songidx]:]
    return h5.root.analysis.tatums_start[h5.root.analysis.songs.cols.idx_tatums_start[songidx]:
                                         h5.root.analysis.songs.cols.idx_tatums_start[songidx+1]]

def get_tatums_confidence(h5,songidx=0):
    """
    Get tatums confidence array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.analysis.songs.nrows == songidx + 1:
        return h5.root.analysis.tatums_confidence[h5.root.analysis.songs.cols.idx_tatums_confidence[songidx]:]
    return h5.root.analysis.tatums_confidence[h5.root.analysis.songs.cols.idx_tatums_confidence[songidx]:
                                              h5.root.analysis.songs.cols.idx_tatums_confidence[songidx+1]]

def get_artist_mbtags(h5,songidx=0):
    """
    Get artist musicbrainz tag array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.musicbrainz.songs.nrows == songidx + 1:
        return h5.root.musicbrainz.artist_mbtags[h5.root.musicbrainz.songs.cols.idx_artist_mbtags[songidx]:]
    return h5.root.musicbrainz.artist_mbtags[h5.root.metadata.songs.cols.idx_artist_mbtags[songidx]:
                                             h5.root.metadata.songs.cols.idx_artist_mbtags[songidx+1]]

def get_artist_mbtags_count(h5,songidx=0):
    """
    Get artist musicbrainz tag count array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.musicbrainz.songs.nrows == songidx + 1:
        return h5.root.musicbrainz.artist_mbtags_count[h5.root.musicbrainz.songs.cols.idx_artist_mbtags[songidx]:]
    return h5.root.musicbrainz.artist_mbtags_count[h5.root.metadata.songs.cols.idx_artist_mbtags[songidx]:
                                                   h5.root.metadata.songs.cols.idx_artist_mbtags[songidx+1]]

def get_year(h5,songidx=0):
    """
    Get release year from a HDF5 song file, by default the first song in it
    """
    return h5.root.musicbrainz.songs.cols.year[songidx]
 


In [0]:
def apply_to_all_files(basedir,func=lambda x: x,ext='.h5'):
    """
    From a base directory, go through all subdirectories,
    find all files with the given extension, apply the
    given function 'func' to all of them.
    If no 'func' is passed, we do nothing except counting.
    INPUT
       basedir  - base directory of the dataset
       func     - function to apply to all filenames
       ext      - extension, .h5 by default
    RETURN
       number of files
    """
    cnt = 0
    # iterate over all files in all subdirectories
    for root, dirs, files in os.walk(basedir):
        files = glob.glob(os.path.join(root,'*'+ext))
        # count files
        cnt += len(files)
        # apply function to all files
        for f in files :
            func(f)       
    return cnt

In [0]:
def strtimedelta(starttime,stoptime):
    return str(datetime.timedelta(seconds=stoptime-starttime))

### Extract non Audio

In [0]:
all_song_id = []
all_artist_id=[]
all_artist_name=[]
all_release=[]
all_title=[]
all_danceability=[]
all_track_id=[]
all_year=[]
all_similar_artists=[]
all_artist_terms=[]
all_artist_terms_freq=[]
all_artist_terms_weight =[]

def nonAudio(filename):
    h5 = open_h5_file_read(filename)
    tmp = get_song_id(h5)
    all_song_id.append(tmp)
    tmp = get_artist_id(h5)
    all_artist_id.append(tmp)
    tmp = get_artist_name(h5)
    all_artist_name.append(tmp)
    tmp = get_release(h5)
    all_release.append(tmp)
    tmp = get_title(h5)
    all_title.append(tmp)
    tmp = get_danceability(h5)
    all_danceability.append(tmp)
    tmp = get_track_id(h5)
    all_track_id.append(tmp)
    tmp = get_year(h5)
    all_year.append(tmp)
    tmp = get_similar_artists(h5)
    all_similar_artists.append(tmp)
    tmp = get_artist_terms(h5)
    all_artist_terms.append(tmp)
    tmp = get_artist_terms_freq(h5)
    all_artist_terms_freq.append(tmp)
    tmp = get_artist_terms_weight(h5)
    all_artist_terms_weight.append(tmp)
    h5.close()

In [301]:
count1 = 0
count2 = 0
for i in all_danceability:
    if i == 0:
        count1+=1
    else:
        count2+=1
    
print(count1,count2)

0 0


Change *smallset* to* msd_subset_data_path* to apply function to whole subset $\rightarrow$ will take ages

In [302]:
t1 = time.time()
apply_to_all_files(smallset,func=nonAudio)
t2 = time.time()
print(t2-t1)

1.1779732704162598


In [0]:
nonAudio = [all_artist_id,
all_artist_name,
all_release,
all_title,
all_danceability,
all_track_id,
all_year,
all_similar_artists,
all_artist_terms,
all_artist_terms_freq,
all_artist_terms_weight]

nonAudioLabels = ['artist_id',
'artist_name',
'release',
'title',
'danceability',
'track_id',
'year',
'similar_artists',
'artist_terms',
'artist_terms_freq',
'artist_terms_weight']

dataNonAudio = pd.DataFrame(nonAudio,index = nonAudioLabels, columns = all_song_id)
nonAudioData = dataNonAudio.transpose()

In [151]:
nonAudioData.head()

,artist_id,artist_name,release,title,danceability,track_id,year,similar_artists,artist_terms,artist_terms_freq,artist_terms_weight
b'SOBLFFE12AF72AA5BA',b'ARJNIUY12298900C91',b'Adelitas Way',b'Adelitas Way',b'Scream',0,b'TRAAAAK128F9318786',2009,"[b'AROQMHX11C8A42BD89', b'ARZEKVZ1187FB51D85',...","[b'alternative metal', b'heavy metal', b'rock'...","[0.9958746574225641, 0.9958746574225641, 1.0, ...","[1.0, 0.8993537393186563, 0.7561876828863154, ..."
b'SOELQTU12AB018949D',b'ARNZW6T1187B989D0F',b'Stanley Black',b'In Memoriam',b'Andalucia',0,b'TRAAADT12903CCC339',2003,"[b'ARKXLIJ1187B9A4C54', b'ARJUNZD1187B9A5DAE',...","[b'easy listening', b'exotica', b'jazz', b'spa...","[0.9731502768474002, 0.8603204163852175, 1.0, ...","[1.0, 0.9828722637802467, 0.9214406530821202, ..."
b'SOFRDWL12A58A7CEF7',b'AR9Q9YC1187FB5609B',b'Quest_ Pup_ Kevo',b'A Bad Azz Mix Tape IV',b'Hit Da Scene',0,b'TRAAACN128F9355673',0,"[b'ARUPWVD1187FB4DA36', b'ARUYGUB11C8A415A47',...","[b'hard trance', b'trip hop', b'progressive tr...","[0.9405512544992299, 0.9522723628927412, 0.982...","[1.0, 0.9946680524617372, 0.9911481839487435, ..."
b'SOCSNVI12A8C13ECC2',b'ARC73361187B9B42BD',b'Big Brother & The Holding Company',b'Be a Brother',b'Heartache People',0,b'TRAAADJ128F4287B47',1970,"[b'ARUP0001187FB39B04', b'ARG1FAQ1187B98FEBD',...","[b'blues-rock', b'italian disco', b'folk rock'...","[0.9684216441460131, 0.9290450934239187, 0.941...","[1.0, 0.9678354486948888, 0.9201864958271878, ..."
b'SOCIWDW12A8C13D406',b'ARMJAGH1187FB546F3',b'The Box Tops',b'Dimensions',b'Soul Deep',0,b'TRAAABD128F429CF47',1969,"[b'ARSZWK21187B9B26D7', b'ARLDW2Y1187B9B544F',...","[b'blue-eyed soul', b'pop rock', b'blues-rock'...","[1.0, 0.8931999928346939, 0.7860602885494408, ...","[1.0, 0.8459884034332037, 0.8306895698215381, ..."


###Extract Audio

In [0]:
all_analysis_sample_rate = []
all_tempo = []
all_time_signature = []
all_segments_pitches= []
all_segments_timbre= []
all_segments_loudness_max= []
all_song_id = []

def extractAudio(filename): 
  
    h5 = open_h5_file_read(filename)
    tmp = get_song_id(h5)
    all_song_id.append(tmp)
    tmp = get_analysis_sample_rate(h5)
    all_analysis_sample_rate.append(tmp)
    tmp = get_time_signature(h5)
    all_time_signature.append(tmp)
    tmp = get_tempo(h5)
    all_tempo.append(tmp)
    tmp = get_segments_pitches(h5)
    all_segments_pitches.append(tmp)
    tmp = get_segments_timbre(h5) 
    all_segments_timbre.append(tmp)
    tmp = get_segments_loudness_max(h5)
    all_segments_loudness_max.append(tmp)
    h5.close()    

run for a small subset

In [15]:
t1 = time.time()
apply_to_all_files(msd_subset_data_path,func=extractAudio)
t2 = time.time()

KeyboardInterrupt: ignored

run for the whole subset

In [0]:
t1 = time.time()
apply_to_all_files(msd_subset_data_path,func=extractAudio)
t2 = time.time()

Extracting the features

In [0]:
Audio = [
    all_analysis_sample_rate,
    all_time_signature,
    all_tempo,
    all_segments_pitches,
    all_segments_timbre, #pure mfcc
    all_segments_loudness_max
]
AudioLabels = [
    'analysis_sample_rate',
    'time_signature',
    'tempo',
    'segments_pitches',
    'segments_timbre', #pure mfcc
    'segments_loudness_max'
]

In [0]:
dataAudio = pd.DataFrame(Audio,index = AudioLabels, columns = all_song_id)
audioData = dataAudio.transpose()

In [152]:
audioData.head()

NameError: ignored

## Get training and test indices
Use **get_all_tags(nonAudioData)** to get all possible tags \\



In [153]:
nonAudioData.head()

,artist_id,artist_name,release,title,danceability,track_id,year,similar_artists,artist_terms,artist_terms_freq,artist_terms_weight
b'SOBLFFE12AF72AA5BA',b'ARJNIUY12298900C91',b'Adelitas Way',b'Adelitas Way',b'Scream',0,b'TRAAAAK128F9318786',2009,"[b'AROQMHX11C8A42BD89', b'ARZEKVZ1187FB51D85',...","[b'alternative metal', b'heavy metal', b'rock'...","[0.9958746574225641, 0.9958746574225641, 1.0, ...","[1.0, 0.8993537393186563, 0.7561876828863154, ..."
b'SOELQTU12AB018949D',b'ARNZW6T1187B989D0F',b'Stanley Black',b'In Memoriam',b'Andalucia',0,b'TRAAADT12903CCC339',2003,"[b'ARKXLIJ1187B9A4C54', b'ARJUNZD1187B9A5DAE',...","[b'easy listening', b'exotica', b'jazz', b'spa...","[0.9731502768474002, 0.8603204163852175, 1.0, ...","[1.0, 0.9828722637802467, 0.9214406530821202, ..."
b'SOFRDWL12A58A7CEF7',b'AR9Q9YC1187FB5609B',b'Quest_ Pup_ Kevo',b'A Bad Azz Mix Tape IV',b'Hit Da Scene',0,b'TRAAACN128F9355673',0,"[b'ARUPWVD1187FB4DA36', b'ARUYGUB11C8A415A47',...","[b'hard trance', b'trip hop', b'progressive tr...","[0.9405512544992299, 0.9522723628927412, 0.982...","[1.0, 0.9946680524617372, 0.9911481839487435, ..."
b'SOCSNVI12A8C13ECC2',b'ARC73361187B9B42BD',b'Big Brother & The Holding Company',b'Be a Brother',b'Heartache People',0,b'TRAAADJ128F4287B47',1970,"[b'ARUP0001187FB39B04', b'ARG1FAQ1187B98FEBD',...","[b'blues-rock', b'italian disco', b'folk rock'...","[0.9684216441460131, 0.9290450934239187, 0.941...","[1.0, 0.9678354486948888, 0.9201864958271878, ..."
b'SOCIWDW12A8C13D406',b'ARMJAGH1187FB546F3',b'The Box Tops',b'Dimensions',b'Soul Deep',0,b'TRAAABD128F429CF47',1969,"[b'ARSZWK21187B9B26D7', b'ARLDW2Y1187B9B544F',...","[b'blue-eyed soul', b'pop rock', b'blues-rock'...","[1.0, 0.8931999928346939, 0.7860602885494408, ...","[1.0, 0.8459884034332037, 0.8306895698215381, ..."


In [0]:
def get_single_tags(df):
    """Returns all songs with single tags"""
    #Get dict of all songs grouped by prevalent tag
    #
    #Drop all songs (rows) from that have 
    print(all_tag_dict)
    single_tag_dict = None
    for tag in tag_dict:
        if len(tag[1]) == 1:
            song_id = tag[1].iloc[0].name
            df.at['song_id', 'prevalent_tag']
            print(df.loc[song_id])
            print('=========================================')

In [0]:
def add_prevalent_tag(df):
    # Adds a column with the highest weighted tag
    df['prevalent_tag'] = df['artist_terms'].apply(lambda tags: tags[0])

    # Get list of all prevalent tags (all_tags)
    all_tags = df['prevalent_tag'].values
    i = 0
    for index, song in df.iterrows():
        # Delete prevalent term of that song from list
        all_tags[i] = None
        # Go through song_artist_terms until artist_term in all_artist_terms
        song_tags = song['artist_terms']
        #print(i, song)
        single = True
        j = 0
        while single and j<len(song_tags):
            new_tag = song_tags[j]
            j += 1
            if new_tag in all_tags:
                single = False
        if single:
            new_tag = b'other'
        df.iloc[i]['prevalent_tag'] = new_tag 
        all_tags[i] = new_tag
        i += 1
        


In [0]:
def get_train_test_split(df, stratify=True):
    """Returns train and test split"""
    if stratify:
        split = train_test_split(df, test_size=0.2, shuffle=True, stratify=df['prevalent_tag'])
    else: 
        split = train_test_split(df, test_size=0.2, shuffle=True)
    return split


In [0]:
def save_train_test_split(df):
    add_prevalent_tag(df)
    train, test = get_train_test_split(df, stratify=True)
    train_indices = list(train.index)
    test_indices = list(test.index)
    np.save('/content/drive/My Drive/IR/Dataprocessing/TrainTestSplit/TrainIndices', train_indices)
    np.save('/content/drive/My Drive/IR/Dataprocessing/TrainTestSplit/TestIndices', test_indices)

In [364]:
print(np.load('/content/drive/My Drive/IR/Dataprocessing/TrainTestSplit/TrainIndices.npy'))

[b'SOERIDA12A6D4F8506' b'SOEPPTV12AC4689D86' b'SOIAZJW12AB01853F1'
 b'SOJROSW12A582A24EC' b'SOUDSGM12AC9618304' b'SOIGHOD12A8C13B5A1'
 b'SOFSOCN12A8C143F5D' b'SOBRKGM12A8C139EF6' b'SOZIULX12A8AE46C39'
 b'SOCSNVI12A8C13ECC2' b'SOFRDWL12A58A7CEF7' b'SOXQUSV12AB018359A'
 b'SOBLFFE12AF72AA5BA' b'SOHOZBI12A8C132E3C' b'SOXVLOJ12AB0189215'
 b'SODFMEL12AC4689D8C' b'SOTAZDY12AB0187616' b'SOCIWDW12A8C13D406'
 b'SOQHXMF12AB0182363' b'SOQPDMB12AB017DECF' b'SOKDFIV12A6D4F9398'
 b'SOHKNRJ12A6701D1F8' b'SOUCVHW12AB018E830' b'SOGSSXS12A8AE4815A'
 b'SOXZYWX12A6310ED0C' b'SOOVHYF12A8C134892' b'SOYKDDB12AB017EA7A'
 b'SONQPZK12AB0182D84' b'SOSMJFC12A8C13DE0C' b'SOKTJDS12AF72A25E5'
 b'SODZYPO12A8C13A91E' b'SOMZWCG12A8C13C480' b'SOIGICF12A8C141BC5'
 b'SOEKAZG12AB018837E' b'SOMTBEP12D021B28AE' b'SOAPERH12A58A787DC'
 b'SONRWUU12AF72A4283' b'SORRNOC12AB017F52B' b'SOTLQMM12AB017EE6D'
 b'SOGVJUR12A8C13485C' b'SOROXIB12A8C135F80' b'SOKBZHG12A81C21426']


In [0]:
df = nonAudioData.copy()

save_train_test_split(df)
#print(nonAudioData.shape)
#add_prevalent_tag2(df)
#get_train_test_split(df)
#df.head()

## Get evaluation playlist
Use **get_all_tags(nonAudioData)** to get all possible tags \\
Use **get_playlist(nonAudioData, tag, max_length)** to get a playlist with desired length sorted by tag_weight for given tag

In [0]:
def get_all_tags(df):
    """Returns a list of all tags/artist_terms"""
    return set(np.concatenate(nonAudioData['artist_terms'].values))

In [0]:
def get_songs_with_tag(df, tag):
    """Returns a DataFrame containing all songs that were tagged with a given
    tag/artist_term."""
    mask = []
    for tags in df['artist_terms'].values:
        if tag in tags:
            mask.append(True)
        else:
            mask.append(False)
    return df[mask]


#example use: get_songs_with_tag(nonAudioData, b'jazz')

In [0]:
def get_tag_weight(song, tag):
    """Returns the tag_weight/artist_term_weight for a given song and a given tag/artist_term"""
    tags = song['artist_terms']
    tag_weights = song['artist_terms_weight']
    return tag_weights[list(tags).index(tag)]

In [0]:
def get_playlist(df, tag, max_length=10):
    """Returns a playlist with desired length sorted by tag_weight/artist_term_weight for a 
    given tag/artist term"""
    # get all songs with the given tag
    playlist = get_songs_with_tag(df, tag)
    # create new column with weight of given tag
    playlist = playlist.assign(weight=playlist.apply(lambda x: get_tag_weight(x, tag), axis=1))
    # sort by tag weight
    playlist = playlist.sort_values(by="weight", ascending=False)
    # clip after max_length songs
    playlist = playlist.iloc[:max_length]
    return playlist

In [20]:
# Example use of get_playlist:
playlist = get_playlist(nonAudioData, b'jazz', max_length=4)
playlist.head()

,artist_id,artist_name,release,title,danceability,track_id,year,similar_artists,artist_terms,artist_terms_freq,artist_terms_weight,weight
b'SOELQTU12AB018949D',b'ARNZW6T1187B989D0F',b'Stanley Black',b'In Memoriam',b'Andalucia',0,b'TRAAADT12903CCC339',2003,"[b'ARKXLIJ1187B9A4C54', b'ARJUNZD1187B9A5DAE',...","[b'easy listening', b'exotica', b'jazz', b'spa...","[0.9731502768474002, 0.8603204163852175, 1.0, ...","[1.0, 0.9828722637802467, 0.9214406530821202, ...",0.921441
b'SOTLQMM12AB017EE6D',b'ARHFSZI11F4C8412BF',b'EST',b'Mostrati',b'Non Lo Saprai Mai',0,b'TRAAAMG128F9318C5E',0,"[b'ARKY0VI1187B9952E1', b'AR8QJHN1187FB3616E',...","[b'nu jazz', b'chill-out', b'jazz fusion', b'f...","[0.9717783236076762, 0.9235330491541319, 0.923...","[1.0, 0.9928987547795262, 0.9755325458406985, ...",0.884649
b'SOOVHYF12A8C134892',b'ARCLYBR1187FB53913',b'Neal Schon',b'Late Nite',"b""I'll Be Waiting""",0,b'TRAAAHD128F42635A5',1989,"[b'ARFBMPN1187B9B0F03', b'AR95I3Q1187FB5AD22',...","[b'instrumental rock', b'smooth jazz', b'hard ...","[0.8362781222636151, 0.8625610363073113, 0.933...","[1.0, 0.9943483517118671, 0.9514865166740996, ...",0.849094
b'SOHOZBI12A8C132E3C',b'AR0MWD61187B9B2B12',b'International Noise Conspiracy',b'Survival Sickness',b'Smash It Up',0,b'TRAAAUR128F428B1FA',2000,"[b'ARJT58W1187B98D43B', b'AR7QOOZ1187B98E1BE',...","[b'ska punk', b'garage rock', b'post-hardcore'...","[0.8846428936490704, 0.9052626213009968, 0.905...","[1.0, 0.972876260863861, 0.9684486212801907, 0...",0.843042
